In [1]:
import pandas as pd
import numpy as np

In [2]:
imdb1 = pd.read_csv("zippedData/imdb.name.basics.csv.gz")
imdb2 = pd.read_csv("zippedData/imdb.title.akas.csv.gz")
imdb3 = pd.read_csv("zippedData/imdb.title.basics.csv.gz")
imdb4 = pd.read_csv("zippedData/imdb.title.crew.csv.gz")
imdb5 = pd.read_csv("zippedData/imdb.title.principals.csv.gz")
imdb6 = pd.read_csv("zippedData/imdb.title.ratings.csv.gz")

In [3]:
imdb4

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943
...,...,...,...
146139,tt8999974,nm10122357,nm10122357
146140,tt9001390,nm6711477,nm6711477
146141,tt9001494,"nm10123242,nm10123248",NaN
146142,tt9004986,nm4993825,nm4993825


# IMDB Web Scraped Data

In [4]:
# cleaning primary keys in imdb dfs
imdb2['title_id'] = imdb2['title_id'].str.strip()
imdb3['tconst'] = imdb3['tconst'].str.strip()
imdb4['tconst'] = imdb4['tconst'].str.strip()
imdb5['tconst'] = imdb5['tconst'].str.strip()
imdb6['tconst'] = imdb6['tconst'].str.strip()

In [5]:
# dropping duplicates
imdb2.drop_duplicates(subset=['title_id'], inplace=True)

In [6]:
imdb3.drop_duplicates(subset=['tconst'], inplace=True)
imdb4.drop_duplicates(subset=['tconst'], inplace=True)
imdb5.drop_duplicates(subset=['tconst'], inplace=True)
imdb6.drop_duplicates(subset=['tconst'], inplace=True)

In [7]:
# importing web scrapes
web_scrape1 = pd.read_csv('imdb_monetary_data_16590.csv')
web_scrape1.set_index(['title_id'], inplace=True)

In [8]:
web_scrape2 = pd.read_csv('imdb_monetary_data_16591_to_25214_.csv')
web_scrape2.set_index(['title_id'], inplace=True)

In [9]:
web_scrape3 = pd.read_csv('imdb_monetary_data_29214_.csv')
web_scrape3.set_index(['title_id'], inplace=True)

In [10]:
web_scrape4 = pd.read_csv('imdb_monetary_data_33736_.csv')
web_scrape4.set_index(['title_id'], inplace=True)

In [11]:
web_scrape5 = pd.read_csv('imdb_monetary_data_49857_.csv')
web_scrape5.set_index(['title_id'], inplace=True)

In [12]:
web_scrape6 = pd.read_csv('imdb_monetary_data_65930_.csv')
web_scrape6.set_index(['title_id'], inplace=True)

In [13]:
web_scrape7 = pd.read_csv('imdb_monetary_data_67456_.csv')
web_scrape7.set_index(['title_id'], inplace=True)

In [14]:
web_scrape8 = pd.read_csv('imdb_monetary_data_77436_.csv')
web_scrape8.set_index(['title_id'], inplace=True)

In [15]:
# concatenating webscraped data
web_scrape = pd.concat([web_scrape1, web_scrape2, web_scrape3, web_scrape4, web_scrape5, web_scrape6, web_scrape7, web_scrape8])
print(web_scrape.columns, end='\n\n')
print(web_scrape.shape, end='\n\n')
print(web_scrape.isna().sum())

Index(['Unnamed: 0', 'Country', 'Budget', 'Opening Weekend USA', 'Gross USA',
       'Cumulative Worldwide Gross', 'Runtime'],
      dtype='object')

(77501, 7)

Unnamed: 0                        0
Country                         178
Budget                        54747
Opening Weekend USA           73279
Gross USA                     72991
Cumulative Worldwide Gross    64313
Runtime                       18117
dtype: int64


In [16]:
# dropping unnamed column
web_scrape.drop(['Unnamed: 0'], axis=1, inplace=True)
web_scrape

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime
title_id,,,,,,
tt0369610,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min
tt0381957,UK,NaN,NaN,NaN,NaN,NaN
tt0401729,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min
tt10010134,France,NaN,NaN,NaN,NaN,52 min
tt10027708,Australia,NaN,NaN,NaN,NaN,94 min
...,...,...,...,...,...,...
tt2997780,India,NaN,NaN,NaN,NaN,105 min
tt2998378,Portugal,NaN,NaN,NaN,"$5,910",90 min
tt2998380,Indonesia,NaN,NaN,NaN,NaN,110 min


In [17]:
# cleaning title_id (index)
web_scrape.index = web_scrape.index.str.strip()

In [18]:
# merging imdb2
df = web_scrape.merge(imdb2, left_on=web_scrape.index, right_on=['title_id'])

In [19]:
df = df.merge(imdb3, left_on=['title_id'], right_on=['tconst'])

In [20]:
df = df.merge(imdb4, left_on=['title_id'], right_on=['tconst'])

In [21]:
df = df.merge(imdb5, left_on=['title_id'], right_on=['tconst'])

In [22]:
df = df.merge(imdb6, how='left', left_on=['title_id'], right_on=['tconst'])

In [23]:
df

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime,title_id,ordering_x,title,region,...,writers,tconst_x,ordering_y,nconst,category,job,characters,tconst_y,averagerating,numvotes
0,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min,tt0369610,10,Джурасик свят,BG,...,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",tt0369610,10,nm0189777,producer,producer,NaN,tt0369610,7.0,539338.0
1,UK,NaN,NaN,NaN,NaN,NaN,tt0381957,1,Cloud Cuckoo Land,GB,...,"nm1458371,nm1461088",tt0381957,10,nm1108038,composer,NaN,NaN,tt0381957,7.6,23.0
2,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min,tt0401729,10,Джон Картер: Мiж двох свiтiв,UA,...,"nm0004056,nm0028764,nm0149290,nm0123194",tt0401729,10,nm0606640,producer,producer,NaN,tt0401729,6.6,241792.0
3,France,NaN,NaN,NaN,NaN,52 min,tt10010134,1,Versailles Rediscovered - The Sun King's Vanis...,NaN,...,"nm2205937,nm1355962,nm10542772,nm0873469",tt10010134,10,nm0674930,editor,NaN,NaN,tt10010134,7.0,5.0
4,Australia,NaN,NaN,NaN,NaN,94 min,tt10027708,1,Miguelito - Canto a Borinquen,NaN,...,nm3495624,tt10027708,10,nm7089722,editor,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77076,India,NaN,NaN,NaN,NaN,105 min,tt2997780,1,Edegarike,IN,...,"nm5746541,nm5746543",tt2997780,10,nm4337423,cinematographer,NaN,NaN,tt2997780,7.3,187.0
77077,Portugal,NaN,NaN,NaN,"$5,910",90 min,tt2998378,1,Famel Top Secret,PT,...,"nm2597242,nm1796990",tt2998378,1,nm2597242,actor,NaN,"[""PP (Pedro Famel)""]",tt2998378,6.0,25.0
77078,Indonesia,NaN,NaN,NaN,NaN,110 min,tt2998380,1,Hello Goodbye,ID,...,nm1907447,tt2998380,10,nm1330371,production_designer,NaN,NaN,tt2998380,6.9,18.0
77079,Canada,NaN,NaN,NaN,NaN,74 min,tt2999132,1,Lobsang,CA,...,nm5747769,tt2999132,1,nm5747768,director,NaN,NaN,NaN,NaN,NaN


In [24]:
# dropping unnecessary columns
df.drop(['ordering_x', 'language', 'attributes', 'original_title', 'Runtime', 'is_original_title', 'tconst_x', 'tconst_y', 'types', 'category', 'job', 'characters', 'ordering_y', 'title'], axis=1, inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77081 entries, 0 to 77080
Data columns (total 16 columns):
Country                       76921 non-null object
Budget                        22728 non-null object
Opening Weekend USA           4221 non-null object
Gross USA                     4508 non-null object
Cumulative Worldwide Gross    13180 non-null object
title_id                      77081 non-null object
region                        68112 non-null object
primary_title                 77081 non-null object
start_year                    77081 non-null int64
runtime_minutes               65427 non-null float64
genres                        75714 non-null object
directors                     75904 non-null object
writers                       62034 non-null object
nconst                        77081 non-null object
averagerating                 47237 non-null float64
numvotes                      47237 non-null float64
dtypes: float64(3), int64(1), object(12)
memory usage: 10.0

# Data Cleaning

In [26]:
# Dropping Null Values
df = df.dropna(subset=['Opening Weekend USA', 'Gross USA', 'Cumulative Worldwide Gross'], how='all')

In [27]:
# Cumulative Worldwide Gross replace NaN with 0
df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].fillna(0)

/Users/davidbruce/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
# Movies from India contain 'INR' substring and need to be converted to USD or dropped
INR = df.loc[df['Cumulative Worldwide Gross'].str.contains('INR') == True]
NPR = df.loc[df['Cumulative Worldwide Gross'].str.contains('NPR') == True]

In [29]:
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('INR') == True].index)

In [30]:
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('NPR') == True].index)

In [31]:
def clean_currency(x):
    if isinstance(x, str):
        x = x.replace('$', '').replace(',', '')
        return x
    return(x)

# df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].str.replace(',', '').str.replace('$', '')

In [32]:
df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].map(lambda x: clean_currency(x))

In [33]:
df['Cumulative Worldwide Gross'] = pd.to_numeric(df['Cumulative Worldwide Gross'])

In [34]:
df['Cumulative Worldwide Gross'][:5]

0     1670401444
2      284139100
8     1025467803
9        2704326
12       1424100
Name: Cumulative Worldwide Gross, dtype: int64

In [35]:
# Gross USA 
df['Gross USA'] = df['Gross USA'].fillna(0)

In [36]:
df['Gross USA'] = df['Gross USA'].map(lambda x: clean_currency(x))

In [37]:
df['Gross USA'] = pd.to_numeric(df['Gross USA'])

In [38]:
df['Gross USA'][:5]

0     652270625
2      73078100
8     334191110
9             0
12            0
Name: Gross USA, dtype: int64

In [39]:
# Opening Weekend USA
df['Opening Weekend USA'] = df['Opening Weekend USA'].fillna(0)

In [40]:
df['Opening Weekend USA'] = df['Opening Weekend USA'].map(lambda x: clean_currency(x))

In [41]:
df['Opening Weekend USA'] = pd.to_numeric(df['Opening Weekend USA'])

In [42]:
df['Opening Weekend USA'][:5]

0     208806270
2      30180188
8     116101023
9             0
12            0
Name: Opening Weekend USA, dtype: int64

In [43]:
df.shape

(13209, 16)

In [44]:
df.Budget

0        $150,000,000
2        $250,000,000
8        $200,000,000
9                 NaN
12                NaN
             ...     
77065             NaN
77068      $5,000,000
77071             NaN
77075             NaN
77077             NaN
Name: Budget, Length: 13209, dtype: object

In [45]:
df['Budget'] = df['Budget'].map(lambda x: clean_currency(x))

In [46]:
df['Budget'] = df['Budget'].fillna('0')

In [47]:
def find_currencies(x):
    currencies = []
    for i in x:
        if i[:3].isalpha():
            currencies.append(i[:3])
    return np.unique(currencies)

In [48]:
budget_curr = df['Budget'].tolist()

In [49]:
list_of_curr = find_currencies(budget_curr).tolist()

In [50]:
list_of_curr

['AMD',
 'ARS',
 'AUD',
 'BGL',
 'BRL',
 'CAD',
 'CHF',
 'CLP',
 'CNY',
 'COP',
 'CZK',
 'DKK',
 'EEK',
 'EGP',
 'EUR',
 'GBP',
 'HKD',
 'HUF',
 'IDR',
 'INR',
 'IRR',
 'ISK',
 'ITL',
 'JPY',
 'KRW',
 'LTL',
 'LVL',
 'MXN',
 'MYR',
 'NGN',
 'NOK',
 'NZD',
 'PEN',
 'PHP',
 'PKR',
 'PLN',
 'RUR',
 'SEK',
 'SGD',
 'THB',
 'TRL',
 'TWD',
 'UAH',
 'VEB',
 'ZAR']

## Switched null values to '0' string in order to apply conversion function

In [51]:
df.Budget.fillna('0', inplace = True)

In [52]:
def conversion_rates(x):
    
    list_of_curr = {'ARS':0.01, 'AUD':0.54, 'BGL':0, 'BRL':0.13, 'CAD':0.55, 'CHF':0.8, 'CLP':0.0009, 'CNY':0.1,
                    'COP':0.000175, 'CZK':0.03, 'DKK':0.11, 'EEK':0, 'EGP':0.04, 'EUR':1.17, 'GBP':0.81, 'HKD':0.08,
                    'HUF':0.002, 'IDR':0.000042, 'INR':0.01, 'ISK':0.0053, 'ITL':0, 'JPY':0.007, 'KRW':0.000633, 'LTL':0.25,
                    'NOK':0.11, 'NZD':0.66, 'PEN':0.28, 'PHP':0.02, 'LVL':1.23, 'MXN':0.03, 'MYR':0.16, 'NGN':0.0026,
                    'PKR':0.006, 'PLN':0.25, 'RUR':0.01, 'SEK':0.11, 'SGD':0.73, 'THB':0.03, 'TRL':0.13, 'TWD':0.03, 
                    'UAH':0.04, 'VEB':0.1, 'ZAR':0.06}
    x = str(x)
    x = x.replace(',', '').replace('$','')
    x = x.lstrip()

    
    if x.isdigit() == True:
        return float(x)
    
    country_code = x[:3]
    if country_code in list_of_curr:
        return (float(x[3:len(list(x))]) * list_of_curr[f'{country_code}'])



## Apply conversion function

In [53]:
df['Budget'] = df['Budget'].apply(lambda x: conversion_rates(x))

In [54]:
df.shape

(13209, 16)

In [55]:
df.isna().sum()

Country                          7
Budget                           4
Opening Weekend USA              0
Gross USA                        0
Cumulative Worldwide Gross       0
title_id                         0
region                        2326
primary_title                    0
start_year                       0
runtime_minutes                765
genres                          55
directors                       36
writers                        956
nconst                           0
averagerating                  343
numvotes                       343
dtype: int64

# Cleaning Genres Column

In [56]:
df['genres'] = df['genres'].fillna('NaN')

In [57]:
df['genres'] = df['genres'].map(lambda x: list(x.split(',')))

In [60]:
df.genres[0][0]

'Action'

# Make df a .csv

In [59]:
# df.to_csv('master_table.csv')